In [1]:
from itertools import chain

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StringType, IntegerType

from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import split, element_at, when, col, count, isnull

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
80,application_1579071174065_0049,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load data from S3

In [3]:
import time
start_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
trip = ss.read.csv("s3a://usf.msds.stillbikeshare/trip.csv", inferSchema=True, header=True)
trip.cache()
weather = ss.read.csv("s3a://usf.msds.stillbikeshare/weather.csv", inferSchema=True, header=True)
weather.cache()
station = ss.read.csv("s3a://usf.msds.stillbikeshare/station.csv", inferSchema=True, header=True)
station.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[id: int, name: string, lat: double, long: double, dock_count: int, city: string, installation_date: string]

# Data Processing

In [5]:
trip_split = trip.withColumn("start_date_split", split("start_date", "(?<=^[^ ]*)\\ "))
trip_split = trip_split.withColumn('start_date_no_time', element_at('start_date_split', 1))
trip_split = trip_split.drop('start_date_split')
trip_split = trip_split.withColumn("zip_code", trip_split["zip_code"].cast(IntegerType()))
weather = weather.withColumn("zip_code", weather["zip_code"].cast(IntegerType()))
weather = weather.withColumnRenamed('zip_code', 'zip_code2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
trip_join= trip_split.withColumn(
    'zip_code_weather',
    when((col("zip_code") >= 94102) & (col("zip_code") <= 94158), 94107)\
    .when((col("zip_code") >= 94035) & (col('zip_code') <= 94043), 94041)\
    .when((col("zip_code") >= 94061) & (col('zip_code') <= 94065), 94063)\
    .when((col("zip_code") >= 94301) & (col('zip_code') <= 94309), 94301)\
    .when((col("zip_code") >= 95101) & (col('zip_code') <= 95196), 95113)
)
trip_join = trip_join.filter(trip_join['zip_code_weather'].isNotNull())
trip_weather = trip_join.join(weather, (trip_join.start_date_no_time == weather.date
                                       ) & (trip_join.zip_code_weather == weather.zip_code2))
trip_weather = trip_weather.withColumnRenamed('id', 'trip_id')
trip_weather.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+------------------+----------------+---------+-----------------+------------------+-----------------+---------------+----------------+---------------+------------+-------------+------------+-----------------------------+------------------------------+-----------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+--------------------+-----------+------+----------------+---------+
|trip_id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|start_date_no_time|zip_code_weather|     date|max_temperature_f|mean_temperature_f|min_temperature_f|max_dew_point_f|mean_dew_point_f|min_dew_point_f|max_humidity|mean_humidity|min_humidity|max_sea_level_pressur

In [7]:
trip_weather_start = trip_weather.join(station, trip_weather.start_station_id == station.id)
trip_weather_start = trip_weather_start.withColumnRenamed('id', 'start_id')\
                                       .withColumnRenamed('name', 'start_name')\
                                       .withColumnRenamed('lat', 'start_lat')\
                                       .withColumnRenamed('long', 'start_long')\
                                       .withColumnRenamed('dock_count', 'start_count')\
                                       .withColumnRenamed('city', 'start_city')\
                                       .withColumnRenamed('installation_date', 'start_installation_date')
trip_weather_station = trip_weather_start.join(station, trip_weather_start.end_station_id == station.id)
trip_weather_station = trip_weather_station.withColumnRenamed('id', 'end_id')\
                                           .withColumnRenamed('name', 'end_name')\
                                           .withColumnRenamed('lat', 'end_lat')\
                                           .withColumnRenamed('long', 'end_long')\
                                           .withColumnRenamed('dock_count', 'end_count')\
                                           .withColumnRenamed('city', 'end_city')\
                                           .withColumnRenamed('installation_date', 'end_installation_date')
trip_weather_station.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+------------------+----------------+---------+-----------------+------------------+-----------------+---------------+----------------+---------------+------------+-------------+------------+-----------------------------+------------------------------+-----------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+--------------------+-----------+------+----------------+---------+--------+--------------------+------------------+-------------------+-----------+-------------+-----------------------+------+--------------------+------------------+-------------------+---------+-------------+---------------------+
|trip_id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscr

In [8]:
print("--- %s seconds ---" % (time.time() - start_time))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- 10.4097580909729 seconds ---

# Write Data to HDFS

In [10]:
trip_weather_station.write.option('path', 's3a://usf.msds.stillbikeshare/merged').saveAsTable('merged')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read Data From HDFS

In [13]:
ss.sql("select * from parquet.`s3a://usf.msds.stillbikeshare/merged`").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+--------------+--------------------+----------------+--------------+--------------------+--------------+-------+-----------------+--------+------------------+----------------+--------+-----------------+------------------+-----------------+---------------+----------------+---------------+------------+-------------+------------+-----------------------------+------------------------------+-----------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+--------------------+-----------+------+----------------+---------+--------+--------------------+------------------+-------------------+-----------+-------------+-----------------------+------+--------------------+------------------+-------------------+---------+-------------+---------------------+
|trip_id|duration|    start_date|  start_station_name|start_station_id|      end_date|    end_station_name|end_station_id|bike_id|subscriptio

# Read Cleaned Data

In [9]:
df = ss.sql("select * from parquet.`s3a://usf.msds.stillbikeshare/clean_data_bike_share`")
df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[trip_id: int, duration: int, start_station_id: int, end_station_id: int, bike_id: int, zip_code: int, zip_code_weather: int, zip_code2: int, start_id: int, start_lat: double, start_long: double, start_count: int, end_id: int, end_lat: double, end_long: double, end_count: int, out_max_wind_Speed_mph: double, out_cloud_cover: double, out_max_temperature_f: double, out_wind_dir_degrees: double, out_mean_visibility_miles: double, out_min_dew_point_f: double, out_max_gust_speed_mph: double, out_mean_sea_level_pressure_inches: double, out_min_temperature_f: double, out_mean_wind_speed_mph: double, out_max_visibility_miles: double, out_mean_temperature_f: double, out_max_dew_point_f: double, out_max_sea_level_pressure_inches: double, out_min_visibility_miles: double, out_mean_humidity: double, out_max_humidity: double, out_min_humidity: double, out_mean_dew_point_f: double, out_min_sea_level_pressure_inches: double, start_station_name: double, end_station_name: double, subscription_

In [11]:
df_drop = df.drop('trip_id', 'start_date', 'bike_id')
df_drop = df_drop.withColumnRenamed('duration', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# RF for full features

In [2]:
df_drop_train = ss.read.parquet("s3a://usf.msds.stillbikeshare/full_train")
df_drop_test = ss.read.parquet("s3a://usf.msds.stillbikeshare/full_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
va = VectorAssembler(outputCol="features", inputCols=df_drop_train.columns[1:]) #except the last col.
lpoints_f_train = va.transform(df_drop_train).select("features", "label")
va = VectorAssembler(outputCol="features", inputCols=df_drop_test.columns[1:]) #except the last col.
lpoints_f_test = va.transform(df_drop_test).select("features", "label")
lpoints_f_train.cache()
lpoints_f_test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

manually create validation set and choose the best set of hyperparameters since n fold cross validation takes forever

In [11]:
f_train, f_valid = lpoints_f_train.randomSplit([0.8, 0.2])
f_train.cache()
f_valid.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [16]:
rf = RandomForestRegressor(maxDepth=5, maxBins= 100, minInstancesPerNode=1, minInfoGain = 0, seed=1)
rfmodel = rf.fit(f_train)

print("RMSE : %s" % RegressionEvaluator().evaluate(rfmodel.transform(f_valid)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy : 4790.850314513121

In [17]:
rf = RandomForestRegressor(maxDepth=10, maxBins= 100, minInstancesPerNode=1, minInfoGain = 0, seed=1)
rfmodel = rf.fit(f_train)

print("RMSE : %s" % RegressionEvaluator().evaluate(rfmodel.transform(f_valid)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy : 4704.447175916294

In [18]:
rf = RandomForestRegressor(maxDepth=20, maxBins= 100, minInstancesPerNode=1, minInfoGain = 0, seed=1)
rfmodel = rf.fit(f_train)

print("RMSE : %s" % RegressionEvaluator().evaluate(rfmodel.transform(f_valid)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy : 4701.5217495017705

In [26]:
print("RMSE : %s" % RegressionEvaluator().evaluate(rfmodel.transform(lpoints_f_test)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE : 4338.903195029896

# Feature Selection

In [28]:
rf = RandomForestRegressor(maxDepth=10, maxBins= 100, minInstancesPerNode=1, minInfoGain = 0, seed=1)
rfmodel = rf.fit(f_train)
attrs = sorted((attr["idx"], attr["name"]) for attr in (chain(*lpoints_f_train
        .schema["features"]
        .metadata["ml_attr"]["attrs"].values())))

feature_scores = [(name, rfmodel.featureImportances[idx]) for idx, name in attrs if rfmodel.featureImportances[idx]]
feature_scores = sorted(feature_scores, key=lambda tup: tup[1], reverse=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
feature_score_selected = [i for i in feature_scores if i[1] >= 0.01]
feature_score_selected = [i[0] for i in feature_score_selected]
feature_score_selected = [i for i in feature_score_selected if not i.startswith('start_weekday') and
                                                             not i.startswith('start_hour') and
                                                             not i.startswith('start_month') and
                                                             not i.startswith('end_weekday') and
                                                             not i.startswith('end_hour') and
                                                             not i.startswith('end_month')]
feature_score_selected = feature_score_selected + ['start_weekday', 'start_hour', 'start_month',
                                                   'end_weekday', 'end_hour', 'end_month', 'label']

feature_score_selected = [i for i in feature_score_selected if i != 'start_station_id' and i != 'end_station_id']
feature_score_selected

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['zip_code', 'start_station_name', 'end_station_name', 'out_cloud_cover', 'out_min_humidity', 'subscription_type', 'out_max_wind_Speed_mph', 'start_name', 'out_max_temperature_f', 'end_name', 'out_mean_sea_level_pressure_inches', 'out_max_dew_point_f', 'out_wind_dir_degrees', 'out_mean_wind_speed_mph', 'out_mean_temperature_f', 'out_min_dew_point_f', 'out_max_gust_speed_mph', 'out_max_sea_level_pressure_inches', 'start_long', 'start_id', 'start_weekday', 'start_hour', 'start_month', 'end_weekday', 'end_hour', 'end_month', 'label']

# RF for selected features

In [22]:
df_selected_train = ss.read.parquet("s3a://usf.msds.stillbikeshare/selected_train")
df_selected_test = ss.read.parquet("s3a://usf.msds.stillbikeshare/selected_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
va = VectorAssembler(outputCol="features", inputCols=df_selected_train.columns[0:-1]) #except the last col.
selected_train = va.transform(df_selected_train).select("features", "label")
selected_test = va.transform(df_selected_test).select("features", "label")
rf = RandomForestRegressor(maxDepth=20, seed=1, maxBins=100)
model_selected = rf.fit(selected_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
print("RMSE : %s" % RegressionEvaluator().evaluate(model_selected.transform(selected_test)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE : 4369.50640375176

# Removed Outliers

In [4]:
df_selected_train = ss.read.parquet("s3a://usf.msds.stillbikeshare/noutlier_train")
df_selected_test = ss.read.parquet("s3a://usf.msds.stillbikeshare/noutlier_test")
df_selected_train.cache()
df_selected_test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[zip_code: int, start_station_name: double, end_station_name: double, out_cloud_cover: double, out_min_humidity: double, subscription_type: double, out_max_wind_Speed_mph: double, start_name: double, out_max_temperature_f: double, end_name: double, out_mean_sea_level_pressure_inches: double, out_max_dew_point_f: double, out_wind_dir_degrees: double, out_mean_wind_speed_mph: double, out_mean_temperature_f: double, out_min_dew_point_f: double, out_max_gust_speed_mph: double, out_max_sea_level_pressure_inches: double, start_long: double, start_id: int, start_weekday: vector, start_hour: vector, start_month: vector, end_weekday: vector, end_hour: vector, end_month: vector, label: int]

In [5]:
va = VectorAssembler(outputCol="features", inputCols=df_selected_train.columns[0:-1]) #except the last col.
selected_train = va.transform(df_selected_train).select("features", "label")
selected_test = va.transform(df_selected_test).select("features", "label")
rf = RandomForestRegressor(maxDepth=20, seed=1, maxBins=100)
model_selected = rf.fit(selected_train)
print("RMSE : %s" % RegressionEvaluator().evaluate(model_selected.transform(selected_test)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE : 514.6049780844372